# Automatically download FESOM2 data from Zenodo based on catalog

## Imports and paths

In [ ]:
import intake
import os
from pathlib import Path
from esmvfc_cattools import download_zenodo_files_for_entry
import logging

In [ ]:
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

In [ ]:
# parameters
data_path = Path("../esm_vfc_data/").resolve()
catalog_file = Path("../catalogs/FESOM2_PI_MESH.yaml")

In [ ]:
os.environ["ESM_VFC_DATA_DIR"] = str(data_path)

## Open catalog and download all data

In [ ]:
cat = intake.open_catalog(str(catalog_file))

In [ ]:
list(cat)

In [ ]:
for entry in [cat[name] for name in cat]:
    download_zenodo_files_for_entry(
        entry,
        force_download=False
    )

## Read catalog entries

In [ ]:
print(cat["FESOM2_sample"].read())

In [ ]:
download_zenodo_files_for_entry(cat["MESH_AUX3D"])

In [ ]:
cat["MESH_AUX3D"].cache[0].clear_all()  # see https://github.com/intake/intake/issues/499
print(cat["MESH_AUX3D"].read())

In [ ]:
cat["MESH_NOD2D"].cache[0].clear_all()  # see https://github.com/intake/intake/issues/499
print(cat["MESH_NOD2D"].read())

In [ ]:
cat["MESH_ELEM2D"].cache[0].clear_all()  # see https://github.com/intake/intake/issues/499
print(cat["MESH_ELEM2D"].read())